# 練習環境  
- 限定されたサンプルを扱うなので、全サンプルから使えるサンプルだけのdatasetを作成する
- TrainのDatasetの各ＲＧＢチャンネルの平均と分散をとっておく

In [31]:
import pandas as pd
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Meiryo'

In [32]:
train_fn = r'./backup/dataset_cur_train.csv'
valid_fn = r'./backup/dataset_cur_valid.csv'
os.path.exists(train_fn) , os.path.exists(valid_fn)

(True, True)

In [33]:
df = pd.read_csv(train_fn,index_col=0)
ex = df['filepath'].map(os.path.exists)
save_fn = os.path.basename(train_fn)
df = df.loc[ex]
df.to_csv(save_fn)
print(save_fn , df.shape)

dataset_cur_train.csv (3370, 5)


In [34]:
df = pd.read_csv(valid_fn,index_col=0)
ex = df['filepath'].map(os.path.exists)
save_fn = os.path.basename(valid_fn)
df = df.loc[ex]
df.to_csv(save_fn)
print(save_fn , df.shape)

dataset_cur_valid.csv (369, 5)


### 画素の分布を確認

In [35]:
train_fn = r'dataset_cur_train.csv'
valid_fn = r'dataset_cur_valid.csv'

In [36]:
df = pd.read_csv(train_fn,index_col=0)
display(df.shape , df.head(3))

(3370, 5)

,red_diff,remain_ends,last_stone_is_red,red_posiotn,filepath
3,-2.0,5,True,-2.0,./dataset_o\WWCC2022_ResultsBook\geme536end4.png
0,2.0,8,True,0.0,./dataset_o\WMCC2023_ResultsBook\geme271end1.png
8,-3.0,0,False,4.0,./dataset_o\WMCC2023_ResultsBook\geme346end9.png


In [88]:
srcimages = []
for path in df['filepath'].values:
    image = Image.open(path)
    image_data = np.array(image)
    normalized_image = image_data.astype(np.float32)
    srcimages.append(normalized_image)
mean = np.mean(srcimages, axis=(0, 1, 2))
std = np.std(srcimages, axis=(0, 1, 2))
display('mean',mean.astype(float) , 'std',std )
print("ほどんど白なのでＲＧＢ全部1に近いはずなのに...この計算は没。")

'mean'

array([7.86710501, 7.86710501, 7.86710501])

'std'

array([44.877377, 44.877377, 44.877377], dtype=float32)

ほどんど白なのでＲＧＢ全部1に近いはずなのに...この計算は没。


In [91]:
mean1 = np.mean(srcimages[0:100], axis=(0, 1, 2))
mean2 = np.mean(srcimages[100:200], axis=(0, 1, 2))
mean3 = np.mean(srcimages[0:200], axis=(0, 1, 2))
display(mean1 , mean2 , mean3)

array([247.74107, 246.41354, 248.74603], dtype=float32)

array([247.73212, 246.44182, 248.80464], dtype=float32)

array([132.56071, 132.56071, 132.56071], dtype=float32)

#　計算を改良その１

In [93]:
srcimages = []
for path in df['filepath'].values:
    image = Image.open(path)
    image_data = np.array(image)
    normalized_image = image_data.astype(np.float32)
    srcimages.append(normalized_image)
pictar = np.array( srcimages ) 

In [109]:
display(pictar.shape,len(pictar[:,:,:,0]))

display(['R:G:B',pictar[:,:,:,0].std()] )
display([ pictar[:,:,:,0].mean() , pictar[:,:,:,1].mean() , pictar[:,:,:,2].mean() ])
display([ pictar[:,:,:,0].std() , pictar[:,:,:,1].std() ,pictar[:,:,:,2].std() ])

(3370, 540, 300, 3)

3370

['R:G:B', 41.334015]

[240.97672, 238.83188, 246.84894]

[41.334015, 43.989445, 41.610004]

#　計算を改良その2

In [111]:
srcimages = []
for path in df['filepath'].values:
    image = Image.open(path)
    image_data = np.array(image)
    normalized_image = image_data.astype(np.float32) / 255
    srcimages.append(normalized_image)
pictar = np.array( srcimages ) 
display(pictar.shape,len(pictar[:,:,:,0]))

display(['R:G:B',pictar[:,:,:,0].std()] )
display([ pictar[:,:,:,0].mean() , pictar[:,:,:,1].mean() , pictar[:,:,:,2].mean() ])
display([ pictar[:,:,:,0].std() , pictar[:,:,:,1].std() ,pictar[:,:,:,2].std() ])

(3370, 540, 300, 3)

3370

['R:G:B', 0.16211227]

[0.94500345, 0.936601, 0.96805465]

[0.16211227, 0.17250435, 0.16316637]

その２の値は妥当っぽい。ほぼ白だしなので１に近く、分散も低い  
しかし本番では、正方形にするため、余白を作ってさらにセンタークリップを行うので条件は変わる。
本番の変換を模索してみる

In [134]:
from torchvision import transforms
import torchvision
import torch

In [174]:
## 余白なしで計算をしてみる
target_size = 224
transform = transforms.Compose([
    #transforms.Pad(( 240 // 2, 0), fill=255, padding_mode='constant'),  # 左右に余白を追加
    #transforms.Resize(target_size),
    #transforms.CenterCrop(target_size),
    transforms.ToTensor()
])

In [175]:
srcimages = []
for path in df['filepath'].values:
    image = Image.open(path)
    q = transform(image)
    q = q.unsqueeze(dim=0)
    srcimages.append(q)
tensor1 = torch.cat(srcimages)
display(tensor1.shape)
display(
    [tensor1[:,0,:,:].mean().item() ,tensor1[:,1,:,:].mean().item(),tensor1[:,2,:,:].mean().item()],
    [tensor1[:,0,:,:].std().item() ,tensor1[:,1,:,:].std().item(),tensor1[:,2,:,:].std().item()] )

torch.Size([3370, 3, 540, 300])

[0.9453915357589722, 0.9369375109672546, 0.9683912396430969]

[0.16209395229816437, 0.1724938601255417, 0.16317903995513916]

numpyでの計算に一致したので、本番の変換をかけて算出する

In [176]:
## 余白あり
target_size = 224
transform = transforms.Compose([
    transforms.Pad(( 240 // 2, 0), fill=0, padding_mode='constant'),  # 左右に余白を追加
    transforms.Resize(target_size),
    transforms.CenterCrop(target_size),
    transforms.ToTensor()
])
srcimages = []
for path in df['filepath'].values:
    image = Image.open(path)
    q = transform(image)
    q = q.unsqueeze(dim=0)
    srcimages.append(q)
tensor1 = torch.cat(srcimages)
display(tensor1.shape)
display(
    [tensor1[:,0,:,:].mean().item() ,tensor1[:,1,:,:].mean().item(),tensor1[:,2,:,:].mean().item()],
    [tensor1[:,0,:,:].std().item() ,tensor1[:,1,:,:].std().item(),tensor1[:,2,:,:].std().item()] )

torch.Size([3370, 3, 224, 224])

[0.525406002998352, 0.5207120180130005, 0.5381810665130615]

[0.47722673416137695, 0.4751463830471039, 0.48789089918136597]

適当そうなので、自前の学習の時はこれを使う